## Setup

In [ ]:
# make the current working directory the project root directory
import os
PROJECT_ROOT_DIRECTORY = 'senior_project'
while os.path.basename(os.getcwd()) != PROJECT_ROOT_DIRECTORY:
    os.chdir(os.pardir)
print('Current working directory:', os.getcwd())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from stable_baselines3 import DQN
from environments.discrete_env_v1 import PortfolioManagementEnv

## Data and Environment

In [ ]:
train_df = pd.read_csv('data\dow_10_returns_train.csv', index_col=0, parse_dates=True) # TODO
train_df.head()

In [ ]:
TICKERS = ['AXP', 'AAPL', 'BA', 'GS', 'INTC', 'JNJ', 'KO', 'NKE', 'PG', 'DIS']

RETURN_COLS = ['RF_RETURN'] + [f'{ticker}_RETURN' for ticker in TICKERS]

FEATURE_COLS = RETURN_COLS # TODO

In [ ]:
train_env = PortfolioManagementEnv(
    train_df, 
    RETURN_COLS, 
    FEATURE_COLS, 
    window_size = 10, # TODO
    episode_length = 90 # TODO
)

## Files and Folders

<span style="color:red">**(RUN ONCE)**</span>

In [ ]:
# experiment_id = ''
# experiment_id = f'EXPERIMENT_N_{int(time.time())}' # TODO
print(experiment_id)

In [ ]:
# # TODO
# models_dir = f'experiments/models/{experiment_id}'
# log_dir = f'experiments/logs'

# if not os.path.exists(models_dir):
#     os.makedirs(models_dir)

# if not os.path.exists(log_dir):
#     os.makedirs(log_dir)

## Train Model

<span style="color:red">**(RUN ONCE)**</span>

In [ ]:
# # TODO
# model = DQN(
#     'MlpPolicy', 
#     train_env, 
#     verbose = 1, 
#     tensorboard_log = log_dir
# )

# TIMESTEPS = 10_000 # Number of timesteps between saves
# for i in range(1,300):
#     model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=experiment_id)
#     model.save(f'{models_dir}/{TIMESTEPS*i}')

## Evaluate Performance

In [ ]:
# 000_000 --> info
# 000_000 --> info

model_number = 100_000 # TODO
model = DQN.load(f'experiments/models/{experiment_id}/{model_number}')

In [ ]:
def evaluate_env(env, title=''):
    obs, done = env.reset(), False
    while not done:
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)

    env.render()
    plt.title(title)
    env.close()

In [ ]:
train_env_performance = PortfolioManagementEnv(
    train_df, 
    RETURN_COLS, 
    FEATURE_COLS, 
    window_size = 10, # TODO
    episode_length = -1
)

val_df = pd.read_csv('data\dow_10_returns_val.csv', index_col=0, parse_dates=True)
val_env = PortfolioManagementEnv(
    val_df, 
    RETURN_COLS, 
    FEATURE_COLS, 
    window_size = 10, # TODO
    episode_length = -1
)

evaluate_env(train_env_performance, 'Train Data')
evaluate_env(val_env, 'Val Data')

## Compare and Save Results

In [ ]:
results_df = pd.read_csv('experiments/results_df.csv', index_col=0, parse_dates=True)
results_df[f'{experiment_id}_TRAIN'] = train_env_performance.get_portfolio_returns()
results_df[f'{experiment_id}_VAL'] = val_env.get_portfolio_returns()

(1+results_df[~results_df[f'{experiment_id}_TRAIN'].isna()]).cumprod().plot(title='Train Performance', figsize=(12,4));
(1+results_df[~results_df[f'{experiment_id}_TRAIN'].isna()])[f'{experiment_id}_TRAIN'].cumprod().plot(color='black', grid=True);

(1+results_df[~results_df[f'{experiment_id}_VAL'].isna()]).cumprod().plot(title='Validation Performance', figsize=(12,4));
(1+results_df[~results_df[f'{experiment_id}_VAL'].isna()])[f'{experiment_id}_VAL'].cumprod().plot(color='black', grid=True);

In [ ]:
# results_df.to_csv('experiments/results_df.csv') # TODO